Esta implementação é baseada no artigo:
Zhen, L., Li, M., Laporte, G., &#38; Wang, W. (2019). A vehicle routing problem arising in unmanned aerial monitoring.
Computers and Operations Research, 105, 1–11. https://doi.org/10.1016/j.cor.2019.01.001

Abstract: Unmanned aerial vehicles (UAVs) are widely used to perform monitoring tasks both in the military and civilian areas, and the planning of their routes is critical. This study investigates a routing problem in which UAVs monitor a set of areas with different accuracy requirements. This problem is a variant of the classical vehicle routing problem (VRP), where one must determine not only the order in which to visit a set of nodes located in the plane, but also the height at which to visit them, which impacts the accuracy level and the service time. An integer programming model is formulated to optimize flight routes and minimize the total time needed to complete the monitoring tasks. A tabu search metaheuristic is developed for the problem. Extensive numerical experiments are conducted to assess the efficiency of the heuristic.

In [1]:
'''Importa as bibliotecas'''
import random
import time
import gurobipy as grb
import numpy as np
import pandas as pd


def carrega_instancia(numero_instancia):
    '''Função utilizada para carregar a instância'''

    if numero_instancia == 1:
        '''Define os nós no espaço por quais os drones podem se movimentar'''
        N = {0: (0, 0), 1: (0, 25), 2: (0, 50), 3: (0, 75), 4: (0, 100),
             5: (1, 0), 6: (1, 25), 7: (1, 50), 8: (1, 75), 9: (1, 100),
             10: (2, 0), 11: (2, 25), 12: (2, 50), 13: (2, 75), 14: (2, 100),
             15: (3, 0), 16: (3, 25), 17: (3, 50), 18: (3, 75), 19: (3, 100)
             }

        N_sem_zero = {n: N[n] for n in range(1, len(N))}

        H = {0: 50, 1: 100, 2: 300}
        H_sem_zero = {n: N[n] for n in range(1, len(H))}

        '''Áreas monitoradas'''
        A = {0: (43, 17), 1: (80, 82), 2: (26, 27), 3: (36, 71), 4: (26, 35), 5: (22, 33), 6: (62, 87), 7: (26, 49)}

        u = {i: {r: 90 for r in range(len(H))} for i in range(len(N))} #random.randint(10, 30)

        b = {a: {i: {r: 1 for r in range(len(H))} for i in range(len(N))} for a in range(len(A))}

        '''Torna os pontos com altura de 300m com acurácia insuficiente e portanto recebe o valor 0'''
        for a in range(len(A)):
            for i in range(len(N)):
                b[a][i][2] = 0

        return N, N_sem_zero, H, H_sem_zero, A, u, b

    if numero_instancia == 2:

        '''Define os nós no espaço por quais os drones podem se movimentar'''
        N = {
            0: (0, 0), 1: (0, 25), 2: (0, 50), 3: (0, 75), 4: (0, 100),
             5: (1, 0), 6: (1, 25), 7: (1, 50), 8: (1, 75), 9: (1, 100),
             10: (2, 0), 11: (2, 25), 12: (2, 50), 13: (2, 75), 14: (2, 100),
             15: (3, 0), 16: (3, 25), 17: (3, 50), 18: (3, 75), 19: (3, 100),
             20: (34, 71), 21: (8, 28), 22: (62, 72), 23: (63, 17), 24: (62, 53),
            25: (22, 33), 26: (26, 78), 27: (64, 54), 28: (25, 54), 29: (55, 34),
            30: (35, 67), 31: (4, 71), 32: (90, 4)
             }

        '''Define os índices N a partir do índice 1'''
        N_sem_zero = {n: N[n] for n in range(1, len(N))}

        '''Define os índices H'''
        H = {0: 0, 1: 50, 2: 100, 3: 300}
        H_sem_zero = {n: N[n] for n in range(1, len(H))}

        '''Áreas monitoradas'''
        A = {0: (34, 71), 1: (8, 28), 2: (62, 72), 3: (63, 17), 4: (62, 53), 5: (22, 33),
              6: (26, 78), 7: (64, 54), 8: (25, 54), 9: (55, 34), 10: (35, 67)}

        '''Define os tempos de monitoramento em 90 segundos'''
        u = {i: {r: 90 for r in H_sem_zero} for i in N_sem_zero}

        '''Define inicialmente que todos os pontos possuem acurária igual a 1, ou seja, são viáveis'''
        b = {a: {i: {r: 1 for r in H} for i in N} for a in A}

        '''Posteriormente escolhe aleatoriamente pontos onde a acurácia é baixa e que não podem ser utilizados
        para monitorar uma área'''
        for a in range(len(A)):
            for i in range(0, len(N), 2):
                for r in range(0, len(H), 2):
                    b[a][i][r] = random.randint(0, 2)

        return N, N_sem_zero, H, H_sem_zero, A, u, b

def calcula_distancia(i, r, j, s):
    '''Calcula a distancia entre 2 pontos (i,r) -> (j,s) e retorna a distância'''

    dist = np.sqrt((i[0]-j[0])**2 + (i[1]-j[1])**2 + (r-s)**2)
    return dist

'''Cria o dicionário para salvar os resultados'''
resultados = {'Tempos Máximos': [], 'Quantidade_de_drones': [], 'Z': [], 'Tempo Computacional': [], 'Quantidade de restrições': [], 'Quantidade de variáveis': []}

'''Define parâmetros que serão variados para avaliar o modelo'''
total_de_drones = [1, 2, 3, 4, 5]
tempos_maximos = [100, 120, 140, 160, 180, 200, 300, 1000, 2000]

'''Carrega a instância'''
N, N_sem_zero, H, H_sem_zero, A, u, b = carrega_instancia(2)

'''Varia a quantidade de drones e o tempo máximo para avaliar os resultados para a instância'''
for qtd_drones in total_de_drones:
    for Tmax in tempos_maximos:
        inicio = time.time()

        '''Declara o modelo'''
        modelo = grb.Model(name="Drones")

        '''Indices dos drones'''
        K = range(0, qtd_drones)

        '''==========================Adicionar as variáveis=========================='''
        x = modelo.addVars(N, H, N, H, K, vtype=grb.GRB.BINARY, lb=0, name='x')
        y = modelo.addVars(N_sem_zero, H_sem_zero, K, vtype=grb.GRB.BINARY, lb=0, name='y')
        u_ = modelo.addVars(N, H, K, vtype=grb.GRB.CONTINUOUS, lb=0, name='u_')

        '''==========================Adiciona Parametros=========================='''
        t = {(i, r, j, s): calcula_distancia(N[i], H[r], N[j], H[s]) for i in N for r in H for j in N for s in H}

        '''==========================Adiciona Restricoes=========================='''

        modelo.addConstrs((x[i, r, j, s, k] == 0 for i in N for j in N for r in H for s in H for k in K if (i, r) == (j, s)), name='R0')

        modelo.addConstrs((grb.quicksum(b[a][i][r]*y[i, r, k] for k in K for r in H_sem_zero for i in N_sem_zero) >= 1 for a in A), name='R2')
        '''Esta restrição garante que ao menos 1 sonda vigie cada área a em A'''

        modelo.addConstrs((grb.quicksum(y[i, r, k] for k in K) <= 1 for i in N_sem_zero for r in H_sem_zero), name='R3')
        '''Esta restrição garante que apenas 1 sonda pare em um nó (i, r)'''

        modelo.addConstrs((grb.quicksum(x[i, r, j, s, k] for s in H for j in N if i != j and r != s) == y[i, r, k] for i in N_sem_zero for r in H_sem_zero for k in K), name='R4')

        modelo.addConstrs((grb.quicksum(x[0, 0, j, s, k] for s in H for j in N) == 1 for k in K), name='R5')

        modelo.addConstrs((grb.quicksum(x[i, r, 0, 0, k] for i in N for r in H) == 1 for k in K), name='R6')

        modelo.addConstrs((grb.quicksum(x[i, r, j, s, k] for s in H for j in N if i != j and r != s) == grb.quicksum(x[j, s, i, r, k] for s in H for j in N if i != j and r != s) for i in N for r in H for k in K), name='R7')

        modelo.addConstrs(((grb.quicksum(t[i, r, j, s]*x[i, r, j, s, k] for r in H for s in H for i in N for j in N) + grb.quicksum(u[i][r]*y[i, r, k] for r in H_sem_zero for i in N_sem_zero)) <= Tmax for k in K), name='R9')
        '''Esta restrição garante que o tempo que a sonda percorre somado ao tempo que ela fica parada em uma área não exceda o tempo máximo que o drone pode voar'''

        modelo.addConstrs((u_[0, 0, k] == 0 for k in K), name='R12')

        modelo.addConstrs((u_[j, s, k] >= u_[i, r, k] +1 -len(N)*len(H)*(1-x[i, r, j, s, k]) for i in N for r in H for j in N_sem_zero for s in H_sem_zero for k in K if i != j and r != s), name='R13')

        modelo.addConstrs((u_[i, r, k] <= len(N)*len(H)-1 for i in N for r in H for k in K), name='R14')

        '''Declara a função objetivo'''
        modelo.setObjective((grb.quicksum(t[i, r, j, s]*x[i, r, j, s, k] for r in H for s in H for i in N for j in N for k in K) + grb.quicksum(u[i][r]*y[i, r, k] for r in H_sem_zero for i in N_sem_zero for k in K)))

        '''==========================Define a função Objetivo=========================='''
        modelo.ModelSense = grb.GRB.MINIMIZE
        modelo.update()
        modelo.write("instancia_spam.lp")

        '''==========================Limita o tempo de execução=========================='''
        modelo.Params.timeLimit = 100

        '''==========================Otimiza o modelo=========================='''
        modelo.optimize()

        '''Identifica o tempo final'''
        final = time.time()

        '''Salva os resultados'''
        resultados['Tempos Máximos'].append(round(Tmax, 3))
        resultados['Quantidade_de_drones'].append(round(qtd_drones, 3))
        resultados['Tempo Computacional'].append(round(final-inicio, 3))
        resultados['Quantidade de variáveis'].append(round(modelo.NumVars, 3))
        resultados['Quantidade de restrições'].append(round(modelo.NumConstrs, 3))

        if modelo.SolCount > 0:
            resultados['Z'].append(round(modelo.ObjVal, 3))
        else:
            resultados['Z'].append('inf')

print('====================================')
print(resultados)
'''Salva os resultados gerados em um arquivo .csv'''
pd.DataFrame(resultados).to_csv('resultados_drones.csv', index=False, sep=';')

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-12-27
Set parameter TimeLimit to value 100
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9819 rows, 17652 columns and 81312 nonzeros
Model fingerprint: 0x87f37cdd
Variable types: 132 continuous, 17520 integer (17520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 6835 rows and 17616 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.03 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Set parameter TimeLimit to value 100
Gurobi Optimizer version 9.

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 9819 rows, 17652 columns and 81312 nonzeros
Model fingerprint: 0xbcbe93fb
Variable types: 132 continuous, 17520 integer (17520 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 1212.9529393
Presolve removed 3531 rows and 4787 columns
Presolve time: 0.16s
Presolved: 6288 rows, 12865 columns, 60248 nonzeros
Found heuristic solution: objective 1080.3866611
Variable types: 96 continuous, 12769 integer (12769 binary)

Root relaxation: objective 1.420100e+02, 118 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  142.01000    0    4 1080.38666  142.01000  86.9%     -    0s
H    0     0  

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 19531 rows, 35304 columns and 162624 nonzeros
Model fingerprint: 0xbc99047e
Variable types: 264 continuous, 35040 integer (35040 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 1090.7361357
Presolve removed 6869 rows and 9574 columns
Presolve time: 0.33s
Presolved: 12662 rows, 25730 columns, 120688 nonzeros
Found heuristic solution: objective 300.7365473
Variable types: 192 continuous, 25538 integer (25538 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Use crossover to convert LP symmetric solution to basic solution...
Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 1.440100e+02, 189 iterations,

Found heuristic solution: objective 194.0199980

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.49 seconds (0.62 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 194.02 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.940199980004e+02, best bound 1.940199980004e+02, gap 0.0000%
Set parameter TimeLimit to value 100
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 29243 rows, 52956 columns and 243936 nonzeros
Model fingerprint: 0x2296db16
Variable types: 396 continuous, 52560 integer (52560 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 25263 rows and 38557 columns
Presolve time: 2.90s
Presolved: 3980 rows, 14399 columns, 60168 nonzeros

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 38955 rows, 70608 columns and 325248 nonzeros
Model fingerprint: 0x05423132
Variable types: 528 continuous, 70080 integer (70080 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 34667 rows and 68086 columns
Presolve time: 0.31s

Explored 0 nodes (0 simplex iterations) in 0.34 seconds (0.47 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Set parameter TimeLimit to value 100
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 38955 rows, 70608 columns and 325248 nonzeros
Model fingerprint: 0x80762dc0
Variable types: 528 continuous, 70080 integer (70080 binary)
Coeffic

  RHS range        [1e+00, 1e+02]
Presolve removed 33727 rows and 88080 columns
Presolve time: 0.15s

Explored 0 nodes (0 simplex iterations) in 0.18 seconds (0.19 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Set parameter TimeLimit to value 100
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 48667 rows, 88260 columns and 406560 nonzeros
Model fingerprint: 0x6ed950fc
Variable types: 660 continuous, 87600 integer (87600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 29007 rows and 87885 columns
Presolve time: 0.19s

Explored 0 nodes (0 simplex iterations) in 0.24 seconds (0.21 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

M

Best objective 1.980199980004e+02, best bound 1.980199980004e+02, gap 0.0000%
Set parameter TimeLimit to value 100
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 48667 rows, 88260 columns and 406560 nonzeros
Model fingerprint: 0xb764fed8
Variable types: 660 continuous, 87600 integer (87600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 2049.6967863
Presolve removed 17171 rows and 23935 columns
Presolve time: 1.18s
Presolved: 31496 rows, 64325 columns, 301720 nonzeros
Found heuristic solution: objective 1631.6431641
Variable types: 480 continuous, 63845 integer (63845 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Use crossover to convert LP symmetric solution to basic solut